In [ ]:
# Import all necesary libraries
import os
import glob

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm

%matplotlib inline

In [ ]:
def save_df(df, filename):
    filepath = f'./{filename}.csv'

    if os.path.exists(filepath):
        os.remove(filepath)

    df.to_csv(filepath, index=False)

## Note
I am providing the data as a tar file because I am developing this in google colab and it is easier to untar the data for use, this is all of the data I collected in PolicyMap

In [ ]:
# Untar data folder
import tarfile
data_tar_path = './data.tar.gz'

# Set base directory for data folder
data_dir = './data/'

if os.path.exists(data_tar_path):
    with tarfile.open(data_tar_path, 'r:gz') as tar:
        tar.extractall('./')

# Data ingestion and cleaning

In [ ]:
import os
import glob

def combine_csvs(data_dir='data'):
  csv_files = glob.glob(os.path.join(data_dir, '**', '*.csv'), recursive=True)

  dfs = []

  for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Check if df is empty before proceeding
    if df.empty:
        print(f"Warning: {csv_file} is empty. Skipping.")
        continue

    first_col = df.columns[0]

    # Check if first_col exists in df before using it for filtering
    if first_col not in df.columns:
        print(f"Warning: First column '{first_col}' not found in {csv_file}. Skipping filter.")
    else:
      # Make sure 'GeoID' is consistent, case-insensitive and stripped
      # Check if 'GeoID' is actually a column header before trying to filter by it
      if 'GeoID' in df.columns or 'geoid' in df.columns:
         df = df[~df[first_col].astype(str).str.strip().str.lower().isin([first_col.lower(), 'geoid'])]
         df = df.reset_index(drop=True)

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

    # filename = os.path.basename(csv_file)
    # df['source_file'] = os.path.splitext(filename)[0]

    dfs.append(df)

  if not dfs:
    print("No CSV files found to concatenate.")
    return pd.DataFrame() # Return an empty DataFrame if no files were found

  combined_df = pd.concat(dfs, ignore_index=True)
  return combined_df

In [ ]:
df = combine_csvs(data_dir)

# Header row is duplicated so remove the second one
df = df[df['geoid'] != 'GeoID']

# Clean up column names and drop redundant or not needed columns
columns_to_remove = ['geography_type_description',
                     'geography_name',
                     'formatted_geoid',
                     'geographic_vintage',
                     'data_source',
                     'selected_location',
                     'data_time_period']

df.drop(columns=columns_to_remove, inplace=True)
df['year'] = 2021

columns_renames = {
    'percent_of_people_who_drove_to_work': 'pct_drove_to_work',
    'avg._vehicles_per_household': 'avg_vehicles_per_household',
    'pct._of_people_walking_to_work': 'pct_walking_to_work',
    'pct._of_people_who_took_public_transit_to_work': 'pct_transit_to_work',
    'pct._of_people_who_rode_bike_to_work': 'pct_bike_to_work',
    'pedestrian-oriented_road_density': 'pedestrian_oriented_road_density',
    'national_walkability_index': 'national_walkability_index',
    'sits_in_state': 'state'
}

# These are to be more friendly for PowerBI
df = df.rename(columns={
    'jobs_within_45_minutes_auto_travel_time':'jobs_45m_auto',
    'pedestrian_oriented_road_density':'ped_oriented_rd_dens',
    'national_walkability_index':'walk_idx'
})

df.rename(columns=columns_renames, inplace=True)

In [ ]:
agg_dict = {
    'state': 'first',
    'distance_to_transit': 'first',
    'jobs_45m_auto': 'first',
    'walk_idx': 'first',
    'road_network_density': 'first',
    'pedestrian_oriented_road_density': 'first',
    'avg_vehicles_per_household': 'first',
    'pct_drove_to_work': 'first',
    'pct_walking_to_work': 'first',
    'pct_transit_to_work': 'first',
    'population_density': 'first',
    'per_capita_income': 'first',
    'year': 'first'
}

numeric_cols = [
    'distance_to_transit',
    'jobs_45m_auto',
    'walk_idx',
    'road_network_density',
    'pedestrian_oriented_road_density',
    'avg_vehicles_per_household',
    'pct_drove_to_work',
    'pct_walking_to_work',
    'pct_transit_to_work',
    'population_density',
    'per_capita_income',
]

# Apply aggregation
df_consolidated = df.groupby('geoid', as_index=False).agg(agg_dict)

assert df_consolidated['geoid'].nunique() == len(df_consolidated), "Duplicate geoids found"

df = df_consolidated
print(df.columns)

In [ ]:
for col in numeric_cols:
    # Convert the column to numeric, coercing errors to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # Impute missing values with the median of the state group
    df[col] = df.groupby('state')[col].transform(lambda x: x.fillna(x.median()))

# It is easier for me to analyize vehicles per household if it is a whole number
df['avg_vehicles_per_household'] = df['avg_vehicles_per_household'].astype(int)

In [ ]:
# Select only the numeric columns for correlation analysis
numeric_df = df.select_dtypes(include=np.number)

# Drop the 'year' column as it has no correlation with other columns
numeric_df = numeric_df.drop(columns=['year'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=1)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

In [ ]:
# Create quartiles for walkability index and per capita income
df['walkability_quartile'] = pd.qcut(df['walk_idx'], q=4, labels=False, duplicates='drop')
df['affordability_quartile'] = pd.qcut(df['per_capita_income'], q=4, labels=False, duplicates='drop')

# Combine quartiles to create a composite index (example: summing quartiles)
# You might want to weight these differently or use a different combination method
df['walkability_affordability_index'] = df['walkability_quartile'] + df['affordability_quartile']

display(df[['walk_idx', 'walkability_quartile', 'per_capita_income', 'affordability_quartile', 'walkability_affordability_index']].head())
save_df(df, 'rev5')

In [ ]:
# Define features and target
features = ['population_density']
target = 'distance_to_transit'

# Drop rows with missing values in the selected features and target
df_cleaned = df.dropna(subset=features + [target])

# Split data into training and testing sets
X = df_cleaned[features]
y = df_cleaned[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

# You can also print the model coefficients
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")